In [8]:
# 크롤링
import requests

In [9]:
import pandas as pd

In [10]:
from bs4 import BeautifulSoup as bs 
import lxml

In [11]:
#selenium
from selenium import webdriver 
import time
import requests
from selenium.webdriver.common.keys import Keys

In [13]:
path = r'C:\Users\슝\GH\Session04. 데이터 수집과 관리\chromedriver.exe' 

In [14]:
# date_list는 0927 기준 일주일 전까지의 날짜
# html_list는 각 날짜별 html 태그 고유 id 숫자
date_list=['0928','0927','0926','0925','0924','0923','0922']
html_list=['0','112','240','357','467','572','669']

In [15]:
# 뒤에 나올 for문에서 순위 변동 바꿔주는 함수!
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

In [16]:
# 크롤링 하고 데이터프레임 저장까지

driver = webdriver.Chrome(path)
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")


for i in range(7):
    for j in range(4):
        driver.implicitly_wait(4)
        driver.find_element_by_id('btn_'+ html_list[i]).click()

    daily_boxoffice_html = driver.page_source
    parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
    daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":'tbody_'+ html_list[i]})
    movie_list = daily_boxoffice.find_all("tr")


    daily_movies = {'영화명' : [], # key영화명 그옆에는 빈 리스트
    '순위 변동' : [],
    '개봉일' : [],
    '매출액' : [],
    '매출액점유율' : [],
    '관객수' : [],
    '누적관객수' : [],
    '스크린수' : [],
    '상영횟수' : []}


    for movie in movie_list:
        title_rank = movie.find("td").find_next_sibling("td").text
        daily_movies['영화명'].append(title_rank[:-5].strip()) #해당하는 키에다가 붙여주는 것
        daily_movies['순위 변동'].append(title_rank[-4:].strip())
        daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())

    daily_boxoffice = pd.DataFrame(daily_movies)

    daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))

    globals()['daily_boxoffice_{}'.format(date_list[i])] = daily_boxoffice
    
    

In [17]:
# 50개씩 잘 긁어졌는지 확인

print(len(daily_boxoffice_0928))
print(len(daily_boxoffice_0927))
print(len(daily_boxoffice_0926))
print(len(daily_boxoffice_0925))
print(len(daily_boxoffice_0924))
print(len(daily_boxoffice_0923))
print(len(daily_boxoffice_0922))

50
50
50
50
50
50
50


> DB로 저장해서 확인

In [18]:
import sqlite3

In [19]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

# 데베 만들기 (칼럼 데이터형식) 
con.execute("CREATE TABLE IF NOT EXISTS boxoffice(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

In [20]:
# DB에 데이터 프레임 추가

daily_boxoffice_0928.to_sql("boxoffice_0928", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0927.to_sql("boxoffice_0927", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0926.to_sql("boxoffice_0926", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0925.to_sql("boxoffice_0925", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0924.to_sql("boxoffice_0924", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0923.to_sql("boxoffice_0923", con, chunksize=1000, if_exists='replace')
daily_boxoffice_0922.to_sql("boxoffice_0922", con, chunksize=1000, if_exists='replace')

C:\Ana\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [21]:
con.commit()
con.close()

In [22]:
# DB 불러오기
con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0928")

boxoffice_0928 = cursor.fetchall()

In [23]:
# 확인해보기
pd.DataFrame(boxoffice_0928)

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,0,2019-09-25,"1,730,292,400",33.7%,"201,985","497,857","1,064","4,815"
1,1,양자물리학,1,2019-09-25,"964,147,750",18.8%,"110,062","276,818",772,"2,728"
2,2,나쁜 녀석들: 더 무비,-1,2019-09-11,"957,134,740",18.7%,"109,794","4,352,983","1,000","4,023"
3,3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"467,389,790",9.1%,"52,391","154,423",678,"1,930"
4,4,애드 아스트라,0,2019-09-19,"176,407,240",3.4%,"19,959","470,598",493,919
5,5,퍼펙트맨,1,2019-10-02,"147,423,700",2.9%,"17,497","74,206",481,723
6,6,타짜: 원 아이드 잭,-1,2019-09-11,"107,045,860",2.1%,"11,724","2,202,587",405,907
7,7,더 룸,-1,2019-09-25,"89,753,700",1.7%,"10,109","36,459",163,496
8,8,예스터데이,-1,2019-09-18,"72,536,220",1.4%,"8,245","285,551",317,470
9,9,극장판 헬로카봇 : 달나라를 구해줘!,7,2019-09-04,"51,966,600",1.0%,"6,644","569,399",172,241
